In [1]:
import pandas as pd

In [2]:
texts = pd.read_csv('texts_updated.csv')
authors_cities = pd.read_csv('cities_updated.csv')
cities_geodata = pd.read_csv('cities_geodata.csv')
authors_countries = pd.read_csv('countries_updated.csv')
countries_geodata = pd.read_csv('countries_geodata.csv')

In [10]:
cities_geodata

,city_id,city_name,lat,lon,distance_to_klagenfurt
0,0,Aachen,50.776351,6.083862,760.0
1,1,Aarau,47.392715,8.044445,483.0
2,2,Adnet,47.696357,13.130757,149.0
3,3,Adnet bei Salzburg,47.696357,13.130757,149.0
4,4,Alblingen,NaN,NaN,NaN
...,...,...,...,...,...
248,248,Zürich,47.372394,8.542333,446.0
249,249,am Dachstein,48.560886,7.531900,553.0
250,250,in Deutschland,51.254489,7.148873,734.0
251,251,südliches Waldviertel,48.317202,15.203420,200.0


In [13]:
df1 = pd.merge(texts[['text_id', 'date_participation']], authors_cities[['text_id', 'city_id', "city"]], left_on="text_id", right_on="text_id")
df1.city_id = df1.city_id.astype(pd.Int32Dtype()).copy()
df1

,text_id,date_participation,city_id,city
0,0,1977-06-27,248,Zürich
1,1,1977-06-27,152,München
2,2,1977-06-27,23,Berlin
3,3,1977-06-27,238,Wien
4,4,1977-06-27,170,Ottobrunn
...,...,...,...,...
871,837,2020-06-21,33,Braunschweig
872,838,2020-06-21,80,Hamburg
873,839,2020-06-21,120,Köln
874,840,2020-06-21,238,Wien


In [29]:
df2 = pd.merge(df1, cities_geodata, "left", left_on="city", right_on="city_name").drop(columns=['city_id_y', 'city']).rename(columns={'city_id_x': 'city_id'})
df2

,text_id,date_participation,city_id,city_name,lat,lon,distance_to_klagenfurt
0,0,1977-06-27,248,Zürich,47.372394,8.542333,446.0
1,1,1977-06-27,152,München,48.137108,11.575382,266.0
2,2,1977-06-27,23,Berlin,52.517036,13.388860,658.0
3,3,1977-06-27,238,Wien,48.208354,16.372504,235.0
4,4,1977-06-27,170,Ottobrunn,48.064382,11.667504,255.0
...,...,...,...,...,...,...,...
871,837,2020-06-21,33,Braunschweig,52.264658,10.523607,684.0
872,838,2020-06-21,80,Hamburg,53.543764,10.009913,828.0
873,839,2020-06-21,120,Köln,50.938361,6.959974,721.0
874,840,2020-06-21,238,Wien,48.208354,16.372504,235.0


In [30]:
df2.date_participation = pd.to_datetime(df2.date_participation).copy()

In [31]:
df2['year_participation'] = df2.date_participation.dt.year

In [41]:
df3 = df2.groupby(['city_name', 'lat', 'lon', 'distance_to_klagenfurt'])['text_id'].count().sort_values(ascending=False).reset_index().rename(columns={'text_id': 'total_participants'})

In [40]:
df3

,city_name,lat,lon,distance_to_klagenfurt,total_participants
0,Berlin,52.517036,13.388860,658.0,180
1,Wien,48.208354,16.372504,235.0,90
2,München,48.137108,11.575382,266.0,52
3,Zürich,47.372394,8.542333,446.0,43
4,Hamburg,53.543764,10.009913,828.0,39
...,...,...,...,...,...
233,Neumarkt,49.279624,11.459466,363.0,1
234,Neu Meteln,53.763130,11.360983,821.0,1
235,München-Pullach,48.055612,11.521745,264.0,1
236,Mönchengladbach,51.194698,6.435364,768.0,1


In [52]:
df4 = df2.groupby(['city_name', 'lat', 'lon', 'distance_to_klagenfurt', 'year_participation'])['text_id'].count().reset_index().rename(columns={'text_id': 'total_participants'}).sort_values('year_participation')

In [54]:
# total_ratings['cumsum'] = total_ratings['total_ratings'].groupby(total_ratings['movieId']).cumsum()
df4['cumsum'] = df4['total_participants'].groupby(df4['city_name']).cumsum()

In [60]:
df4[['city_name', 'year_participation', 'cumsum']]

,city_name,year_participation,cumsum
100,Bottmingen,1977,1
500,Weissenau,1977,1
505,Wien,1977,3
270,Klagenfurt,1977,1
489,Villach,1977,1
...,...,...,...
222,Hamburg,2020,39
588,Zürich,2020,43
458,Straßengel,2020,1
542,Wien,2020,90


In [63]:
df4[df4.year_participation == 1977][]

,city_name,lat,lon,distance_to_klagenfurt,year_participation,total_participants,cumsum
100,Bottmingen,47.523450,7.574873,520.0,1977,1,1
500,Weissenau,46.671719,7.832655,495.0,1977,1,1
505,Wien,48.208354,16.372504,235.0,1977,3,3
270,Klagenfurt,46.622816,14.307960,0.0,1977,1,1
489,Villach,46.616728,13.850027,35.0,1977,1,1
466,Tamsweg,47.126106,13.810382,67.0,1977,1,1
401,Ottobrunn,48.064382,11.667504,255.0,1977,1,1
346,München,48.137108,11.575382,266.0,1977,3,3
195,Hamburg,53.543764,10.009913,828.0,1977,1,1
556,Zürich,47.372394,8.542333,446.0,1977,1,1


In [62]:
pd.DataFrame({'city_name': df4.city_name.unique(), 'year_participation': [1977] * len(df4.city_name.unique()), 'cumsum': [0] * len(df4.city_name.unique())})

,city_name,year_participation,cumsum
0,Bottmingen,1977,0
1,Weissenau,1977,0
2,Wien,1977,0
3,Klagenfurt,1977,0
4,Villach,1977,0
...,...,...,...
233,Freidorf (Kanton Thurgau),1977,0
234,Hannover,1977,0
235,Braunschweig,1977,0
236,Neu Meteln,1977,0


In [58]:
import plotly.express as px

fig = px.scatter_geo(df4, lat="lat", lon="lon", color="city_name",
                     hover_name="city_name", size="cumsum",
                     animation_frame="year_participation",
                     projection="natural earth")
fig.show()

